# Set up the environment
1. Install the necessary libraries
2. Authenticate with GEE

In [2]:
# !pip install geopandas
# !pip install rasterio
# !pip install fulcrum
# !pip install rasterstats
# !pip install StringIO
# !pip install ee
# !pip install earthengine-api --upgrade
# !pip install importlib
# !pip install open_clip_torch
# !pip install contextily

In [0]:
from io import StringIO
import requests, zipfile, io
import contextily as ctx
from matplotlib_scalebar.scalebar import ScaleBar
import shapely.geometry
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import json
import reportlab
from reportlab.lib.pagesizes import A4
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
import math
import warnings
from datetime import datetime, timedelta
import ee  # import Google earth engine module
import geopandas as gpd
import numpy as np
import pandas as pd
import rasterio
from fulcrum import Fulcrum
from rasterio.transform import from_origin
from rasterstats import zonal_stats

In [111]:

# Authenticate the Google Earth engine with Google account
# First setup only, no need to run this after first run
# ee.Authenticate()
ee.Initialize()
# warnings.filterwarnings('ignore')

In [4]:
with open("credentials.json") as c:
	credentials = json.load(c)

fulcrum = Fulcrum(key=credentials['fulcrum_api'])

In [36]:
# todo: add project name and use throughout the file
projectName = ['BRT'] 

In [6]:
formData = fulcrum.forms.find(credentials['TMO_TI_form'])

recordCount = formData['form']['record_count']
pages = math.ceil(recordCount / 5000)
# way cleaner than my script. Thanks!
data = []
for p in range(1, pages + 1):
	dataPage = fulcrum.records.search(
		url_params={'form_id': credentials['TMO_TI_form'], 'page': p, 'per_page': 5000})['records']
	data.extend(dataPage)

In [7]:
df = pd.DataFrame(data)
dups = df[df.duplicated(subset=['latitude','longitude'],keep='first')]

In [8]:
# simplify data and un-nest values from json. Add all trees without spread get a 10m value.
for record in data:
	if 'f6ef' in record['form_values']:
		record['health'] = ''.join(record['form_values']['f6ef']['choice_values'])
	else:
		record['health'] = 'NaN'

	if '0a3e' in record['form_values']:
		record['structure'] = ''.join(record['form_values']['0a3e']['choice_values'])
	else:
		record['structure'] = 'NaN'

	if '009b' in record['form_values']:
		record['height'] = ''.join(record['form_values']['009b'])
	else:
		record['height'] = 'NaN'

	if '7a77' in record['form_values']:
		record['DBH'] = ''.join(record['form_values']['7a77'])
	else:
		record['DBH'] = 'NaN'

	if '6e5e' in record['form_values']:
		record['source'] = ''.join(record['form_values']['6e5e']['choice_values'])
	else:
		record['source'] = 'NaN'

	if 'b96d' in record['form_values']:
		record['spread'] = ''.join(record['form_values']['b96d'])
	else:
		record['spread'] = '7'

	if '3fee' in record['form_values']:
		record['species'] = ''.join(record['form_values']['3fee']['choice_values'])
	else:
		record['species'] = 'No Name - لا يوجد اسم'
	# genus 
	record['genus'] = record['species'].split(' ')[0]
	if record['genus'] in ['no', '', 'No']:
		record['genus'] = 'No name'

	# class
	if record['genus'] in ['Phoenix', 'Washingtonia']:
		record['Class'] = 'Palm'
	elif record['genus'] == 'No name':
		record['Class'] = 'Not identified'
	else:
		record['Class'] = 'Shade'

	# select dead and missing from inventory and lable them future trees
	if record['status'] in ['Missing tree - لا يوجد شجرة', 'Dead tree - شجرة ميتة', 'Plant tree - ازرع شجرة']:
		record['existing'] = 'no'
	else:
		record['existing'] = 'yes'

	# remove arabic strings can also be done with split string for the entire data set instead of hardcoding stuff
	record['status'] = record['status'].split(' - ')[0]

	# if record['status'] == 'No action required - لا حاجة لأي إجراء':
	# 	record['status'] = 'No action required'
	# elif record['status'] == 'Request of activation - طلب تفعيل':
	# 	record['status'] = 'Request of activation'
	# elif record['status'] == 'Request of inspection (RFI) - طلب معاينة':
	# 	record['status'] = 'Request of inspection'
	# elif record['status'] == 'Escalation issue - قضية تصعيد':
	# 	record['status'] = 'Escalation issue'
	# else:
	# 	record['status'] = 'Issue in place'

# '''
# if record['genus'] in ['Phoenix', 'Washingtonia', 'Acacia', '', '','']:
#         record['Type'] = 'Tree'
#     else:
#         record['Type'] = 'Shrubs'
# '''


In [9]:
# get unique project IDs in a list
projectList = list()
for record in data:
	projectId = record['project_id']
	if projectId not in projectList:
		projectList.append(projectId)

# create dict with matching names
result = {}
for i in projectList:
	try:
		projectInfo = fulcrum.projects.find(i)['project']
		result[i] = projectInfo['name']
	except:
		result[i] = 'Unknown Project'

# add project names to org data
for record in data:
	record['project'] = result.get(record['project_id'], 'Unknown')

In [10]:
df = pd.DataFrame(data)
df.drop(columns=['id', 'form_id', 'created_by_id', 'version', 'id', 'created_at', 'updated_at',
                 'created_by', 'assigned_to_id', 'altitude', 'speed', 'course', 'horizontal_accuracy',
                 'vertical_accuracy', 'updated_by', 'updated_by_id', 'created_location', 'updated_location',
                 'created_duration', 'updated_duration', 'edited_duration', 'project_id', 'record_series_id',
                 'assigned_to', 'form_values'], axis=1, inplace=True)

inventory = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude))
inventory.crs = 'EPSG:4326'
inventory = inventory.to_crs(32638)
inventory.drop_duplicates(['geometry'], inplace=True)

In [ ]:
# inventory['dups']=inventory.duplicated(['geometry'],'first')
# inventory.value_counts(['dups'])
# inventory.plot('dups', 'coolwarm_r', markersize=1,figsize=[10,10])
# inventory.to_file(r"/Users/philipp/Downloads/dups.gpkg", driver='GPKG', layer='TMO_dups')

In [ ]:
# filter by project

In [49]:
projectInventory =inventory[inventory['project'].str.contains('BRT', na = False)]

In [0]:
projectBbox32638 = gpd.GeoDataFrame(geometry=pd.DataFrame(projectInventory.buffer(100).envelope).values.flatten(),
                                    crs=32637)

In [0]:
minx, miny, maxx, maxy = projectInventory.geometry.total_bounds

In [76]:
projectInventory.geometry.total_bounds

array([ 664464.94864397, 2714710.45996134,  684827.11332321,
       2749206.85341744])

In [52]:
# Canopy spread
projectInventory['spread'] = projectInventory.spread.astype(str).astype(float)

projectInventory.loc[projectInventory['spread'] < 7, 'spread'] = 7
projectInventory.loc[projectInventory['spread'].isna(), 'spread'] = 7
projectInventory.reset_index(inplace=True, drop=True)

# Create a buffer per point
projectInventory.loc[projectInventory.geometry.type == 'Point', 'geometry'] = projectInventory.buffer(
	(projectInventory['spread'] / 2))

projectInventoryDis = projectInventory.dissolve()
projectInventoryDis['canopyArea_m2'] = projectInventoryDis.area

# Calculate the sum of areas
canopyArea = sum(projectInventoryDis['canopyArea_m2'])
# projectArea = projectBoundarygdf.area

/Users/philipp/Projects/PycharmProjects/fulcrum-python-tools/venv/lib/python3.9/site-packages/geopandas/geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [0]:
from shapely.geometry import box
  
# total bounds of the projectInventory       
xmin,ymin,xmax,ymax =  projectInventory.total_bounds
print(xmin,ymin,xmax,ymax)

# convert to polygon
geom = box(*projectInventory.total_bounds)

In [118]:
# make GEE feature collection from buffered project boundary bounding box
gdf = gpd.GeoDataFrame(index=[0], crs='epsg:32637', geometry=[geom])
projectBbox32637=gdf.buffer(100)
projectBbox32637 = projectBbox32637.to_crs(4326).to_json()
projectBbox32637 = ee.FeatureCollection(json.loads(projectBbox32637))

In [122]:
gdf = gpd.GeoDataFrame(index=[0], crs='epsg:32637', geometry=[geom])
project4326 = gdf.to_crs(4326).to_json()
project4326 = ee.FeatureCollection(json.loads(project4326))

In [125]:
spread=projectInventory.copy()

In [ ]:
######################## set time frame
today = (datetime.today() - timedelta(days=1)).strftime("%Y-%m-%d")  # number of days as a delimiter
before = (datetime.today() - timedelta(days=20)).strftime("%Y-%m-%d")  # number of days as a delimiter
# days_in_interval = today - before
# days_in_interval = 20

# get Sentinel data
collection = (ee.ImageCollection('COPERNICUS/S2_HARMONIZED')
              .filterDate(before, today)
              .filterBounds(project4326.geometry())
              .map(lambda image: image.clip(project4326.geometry()))
              .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 1))
              .map(lambda image: image.normalizedDifference(['B8', 'B4']).rename('ndvi'))
              )
x1 = str(projectBbox32638.geometry().getInfo()['coordinates'][0][0]).strip("[]").split(',')[0]
y1 = str(projectBbox32638.geometry().getInfo()['coordinates'][0][1]).strip("[]").split(',')[1]
transform = from_origin(float(x1), float(y1), 10, 10)

In [147]:
imgFirst = collection.limit(1, 'system:index').first()
imgLast = collection.limit(1, 'system:index', False).first()

#downloading the images
r = requests.get(imgFirst.getDownloadURL({
	'name': 'imgFirst-' + imgFirst.getInfo()['properties']['system:index'].split("T")[0],
	# 'region': str(projectBbox4326.geometry()),
	# 'dimensions': str(imgFirst.getInfo()['bands'][0]['dimensions'][0])+"x"+str(imgFirst.getInfo()['bands'][0]['dimensions'][1])
}))
#unzip it to the selected directory
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall('SentinelData')

r = requests.get(imgLast.getDownloadURL({
	'name': 'imgLast-' + imgLast.getInfo()['properties']['system:index'].split("T")[0],
	# 'region': str(projectBbox4326.geometry()),
	# 'dimensions': str(imgFirst.getInfo()['bands'][0]['dimensions'][0])+"x"+str(imgFirst.getInfo()['bands'][0]['dimensions'][1])
}))
#unzip it to the selected directory
z = zipfile.ZipFile(io.BytesIO(r.content))

z.extractall('SentinelData')

In [148]:
########### process first image
with rasterio.open("SentinelData/imgFirst-" +imgFirst.getInfo()["properties"]['system:index'].split("T")[0]+".ndvi.tif") as src:
	affine = src.transform
	ndval = src.nodatavals[0]
	array = src.read(1)
	array = array.astype('float64')
	array[array == ndval] = np.nan
	df_zonal_stats = pd.DataFrame(
		zonal_stats(spread, array, stats='max', affine=affine, nodata=-99999999, all_touched=True))

# adding statistics back to original GeoDataFrame
imgResults = pd.concat([spread, df_zonal_stats], axis=1)

############# NDVI health
imgResults.loc[(imgResults['max'] >= 0.5), 'ndvi_health_cat'] = 'good - جيد'
imgResults.loc[(imgResults['max'] >= 0.3) & (imgResults['max'] < 0.5), 'ndvi_health_cat'] = 'fair - معتدل'
imgResults.loc[(imgResults['max'] > 0.2) & (imgResults['max'] < 0.3), 'ndvi_health_cat'] = 'poor - سئ'
imgResults.loc[(imgResults['max'] <= 0.2), 'ndvi_health_cat'] = 'dead - ميت'
imgResults.rename(
	columns={'ndvi_health_cat': 'ndvi_health_cat' + imgFirst.getInfo()['properties']['system:index'].split("T")[0]},
	inplace=True)

imgResults.loc[(imgResults['max'] >= 0.2), 'ndvi_health'] = 1
imgResults.loc[(imgResults['max'] < 0.2), 'ndvi_health'] = 0

imgResults.rename(
	columns={'ndvi_health': 'ndvi_health_' + imgFirst.getInfo()['properties']['system:index'].split("T")[0]},
	inplace=True)
imgResults.rename(columns={'max': 'max_' + imgFirst.getInfo()['properties']['system:index'].split("T")[0]},inplace=True)

In [149]:
############# process last image
with rasterio.open("SentinelData/imgLast-" +imgLast.getInfo()["properties"]['system:index'].split("T")[0]+".ndvi.tif") as src:
	affine = src.transform
	ndval = src.nodatavals[0]
	array = src.read(1)
	array = array.astype('float64')
	array[array == ndval] = np.nan
	df_zonal_stats = pd.DataFrame(
		zonal_stats(spread, array, stats='max', affine=affine, nodata=-99999999, all_touched=True))

# adding statistics back to original GeoDataFrame
imgResults = pd.concat([imgResults, df_zonal_stats], axis=1)
################ NDVI health (new categories due to very negative results, violoating the 0.2 threshold)
imgResults.loc[(imgResults['max'] >= 0.5), 'ndvi_health_cat'] = 'good - جيد'
imgResults.loc[(imgResults['max'] >= 0.3) & (imgResults['max'] < 0.5), 'ndvi_health_cat'] = 'fair - معتدل'
imgResults.loc[(imgResults['max'] > 0.2) & (imgResults['max'] < 0.3), 'ndvi_health_cat'] = 'poor - سئ'
imgResults.loc[(imgResults['max'] <= 0.2), 'ndvi_health_cat'] = 'dead - ميت'

imgResults.rename(
	columns={'ndvi_health_cat': 'ndvi_health_cat' + imgLast.getInfo()['properties']['system:index'].split("T")[0]},
	inplace=True)
imgResults.loc[(imgResults['max'] >= 0.2), 'ndvi_health'] = 1
imgResults.loc[(imgResults['max'] < 0.2), 'ndvi_health'] = 0

imgResults.rename(
	columns={'ndvi_health': 'ndvi_health_' + imgLast.getInfo()['properties']['system:index'].split("T")[0]},
	inplace=True)
imgResults.rename(columns={'max': 'max_' + imgLast.getInfo()['properties']['system:index'].split("T")[0]}, inplace=True)

########################################## diff
imgResults['diff'] = imgResults['ndvi_health_' + imgLast.getInfo()['properties']['system:index'].split("T")[0]] - \
                     imgResults['ndvi_health_' + imgFirst.getInfo()['properties']['system:index'].split("T")[0]]
#print(zsresults['diff'].value_counts())

# print(lastimg.getInfo()['properties']['system:index'].split("T")[0])
# print(imgfirst.getInfo()['properties']['system:index'].split("T")[0])
# print(zsresults['ndvi_health_'+lastimg.getInfo()['properties']['system:index'].split("T")[0]])
# print(zsresults['ndvi_health_cat'+lastimg.getInfo()['properties']['system:index'].split("T")[0]].value_counts())


# if health older than 6 month use NDVI health
imgResults['health_merge'] = imgResults['health']
imgResults.loc[(imgResults['health_merge'] == 'NaN'), 'health_merge'] = imgResults[
	'ndvi_health_cat' + imgLast.getInfo()['properties']['system:index'].split("T")[0]]
imgResults.loc[(imgResults['health_merge'].isna()), 'health_merge'] = imgResults[
	'ndvi_health_cat' + imgLast.getInfo()['properties']['system:index'].split("T")[0]]

####print
print(imgResults.value_counts(['health_merge']))
print(imgResults.isna().value_counts(['health_merge']))

###save to file
#imgResults.to_file(r"/content/drive/MyDrive/BPLA/20230510-DQ-inv-ml-ndvi-health-1.gpkg", driver='GPKG', layer='DQ-inv-ml-ndvi-volume-corrected-74-2')
for i in range(len(imgResults)):
	if imgResults.loc[i, 'health_merge'] == 'good - جيد':
		imgResults.loc[i, 'health_updated'] = 'Good'
	elif imgResults.loc[i, 'health_merge'] == 'fair - معتدل':
		imgResults.loc[i, 'health_updated'] = 'Moderate'
	elif imgResults.loc[i, 'health_merge'] == 'poor - سئ':
		imgResults.loc[i, 'health_updated'] = 'Poor'
	elif imgResults.loc[i, 'health_merge'] == 'excellent - ممتاز':
		imgResults.loc[i, 'health_updated'] = 'Excellent'
	elif imgResults.loc[i, 'health_merge'] == 'dead - ميت':
		imgResults.loc[i, 'health_updated'] = 'Dead'
	else:
		imgResults.loc[i, 'health_updated'] = 'Missing'
# todo: stringsplit 

health_merge
dead - ميت      6349
Name: count, dtype: int64
health_merge
False           6349
Name: count, dtype: int64


In [0]:
imgResults.plot('health_merge')

In [152]:
imgResults.to_file('BRT-ndvi.gpkg', driver='GPKG', layer='name')  